# PLN Detección de Spams

Datos: https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('spam').getOrCreate()

In [0]:
ruta = 'dbfs:/FileStore/shared_uploads/jgamarramoreno@gmail.com/SMSSpamCollection'

In [0]:
datos = spark.read.csv(ruta,inferSchema=True,sep='\t')

In [0]:
datos = datos.withColumnRenamed('_c0','clase').withColumnRenamed('_c1','texto')

In [0]:
datos.show()

+-----+--------------------+
|clase|               texto|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



### Preparar los datos

In [0]:
from pyspark.sql.functions import length

In [0]:
datos = datos.withColumn('longitud',length(datos['texto']))

In [0]:
datos.show()

+-----+--------------------+--------+
|clase|               texto|longitud|
+-----+--------------------+--------+
|  ham|Go until jurong p...|     111|
|  ham|Ok lar... Joking ...|      29|
| spam|Free entry in 2 a...|     155|
|  ham|U dun say so earl...|      49|
|  ham|Nah I don't think...|      61|
| spam|FreeMsg Hey there...|     147|
|  ham|Even my brother i...|      77|
|  ham|As per your reque...|     160|
| spam|WINNER!! As a val...|     157|
| spam|Had your mobile 1...|     154|
|  ham|I'm gonna be home...|     109|
| spam|SIX chances to wi...|     136|
| spam|URGENT! You have ...|     155|
|  ham|I've been searchi...|     196|
|  ham|I HAVE A DATE ON ...|      35|
| spam|XXXMobileMovieClu...|     149|
|  ham|Oh k...i'm watchi...|      26|
|  ham|Eh u remember how...|      81|
|  ham|Fine if thats th...|      56|
| spam|England v Macedon...|     155|
+-----+--------------------+--------+
only showing top 20 rows



In [0]:
datos.groupby('clase').count().show()

+-----+-----+
|clase|count|
+-----+-----+
|  ham| 4827|
| spam|  747|
+-----+-----+



### Transformación de Características

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer

In [0]:
tokenizador = Tokenizer(inputCol="texto",outputCol="tokens")
filtro = StopWordsRemover(inputCol='tokens',outputCol='tokens_filtrado')
vectorizador_conteo = CountVectorizer(inputCol='tokens_filtrado',outputCol='vec_conteo')
idf = IDF(inputCol='vec_conteo',outputCol="tf_idf")
clase_a_num = StringIndexer(inputCol='clase',outputCol='label')

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [0]:
datos_listos = VectorAssembler(inputCols=["tf_idf","longitud"],
                               outputCol='features')

### Modelo de Predicción

Usaremos Naive Bayes en este ejemplo

In [0]:
from pyspark.ml.classification import NaiveBayes

In [0]:
nb = NaiveBayes()

### Canalización

In [0]:
from pyspark.ml import Pipeline

In [0]:
secuencia = [clase_a_num,tokenizador,filtro,vectorizador_conteo,idf,datos_listos]

In [0]:
canal_prep_datos = Pipeline(stages=secuencia)

In [0]:
limpiador = canal_prep_datos.fit(datos)

In [0]:
datos_limpios = limpiador.transform(datos)

In [0]:
datos_limpios.show()

+-----+--------------------+--------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|clase|               texto|longitud|label|              tokens|     tokens_filtrado|          vec_conteo|              tf_idf|            features|
+-----+--------------------+--------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|     111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|      29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
| spam|Free entry in 2 a...|     155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
|  ham|U dun say so earl...|      49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|

### Entrenamiento y Evaluación

In [0]:
datos_limpios = datos_limpios.select(['label','features'])

In [0]:
datos_limpios.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [0]:
(dat_ent,dat_pru) = datos_limpios.randomSplit([0.7,0.30])

In [0]:
predictor_spams = nb.fit(dat_ent)

In [0]:
datos.printSchema()

root
 |-- clase: string (nullable = true)
 |-- texto: string (nullable = true)
 |-- longitud: integer (nullable = true)



In [0]:
resultados_test = predictor_spams.transform(dat_pru)

In [0]:
resultados_test.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-798.08519567733...|[1.0,1.1198448668...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-569.75016539767...|[1.0,1.7087549491...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-830.69269936738...|[1.0,3.6479603616...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-801.21216625024...|[1.0,1.5651910748...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-875.76705846708...|[1.0,1.5496671808...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-217.39406062744...|[1.0,4.5363921754...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-841.67141670791...|[1.0,1.1149993811...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-989.60296258171...|[0.99999999999964...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-1010.8829804148...|[1.0,4.3631658926...|       0.0|
|  0.0|(13424,[0

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
eval_acc = MulticlassClassificationEvaluator()

In [0]:
acc = eval_acc.evaluate(resultados_test)

In [0]:
print("La exactitud del modelo al predecir spams fue:%.2f%%"%(acc*100))

La exactitud del modelo al predecir spams fue:92.47%
